## Chess Grandmaster Prediction

Data source: https://www.kaggle.com/datasets/vikasojha98/top-women-chess-players

### Getting Started 

In [49]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [2]:
data = pd.read_csv('top_women_chess_players_aug_2020.csv')
data

,Fide id,Name,Federation,Gender,Year_of_birth,Title,Standard_Rating,Rapid_rating,Blitz_rating,Inactive_flag
0,700070,"Polgar, Judit",HUN,F,1976.0,GM,2675,2646.0,2736.0,wi
1,8602980,"Hou, Yifan",CHN,F,1994.0,GM,2658,2621.0,2601.0,NaN
2,5008123,"Koneru, Humpy",IND,F,1987.0,GM,2586,2483.0,2483.0,NaN
3,4147103,"Goryachkina, Aleksandra",RUS,F,1998.0,GM,2582,2502.0,2441.0,NaN
4,700088,"Polgar, Susan",HUN,F,1969.0,GM,2577,NaN,NaN,wi
...,...,...,...,...,...,...,...,...,...,...
8548,3302288,"Reinkens, Natalia",BOL,F,NaN,NaN,1801,NaN,NaN,wi
8549,343960,"Saffova, Michaela",CZE,F,1994.0,NaN,1801,1791.0,1765.0,NaN
8550,5038294,"Shetye, Siddhali",IND,F,1992.0,NaN,1801,1884.0,1824.0,wi
8551,2072491,"Trakru, Priya",USA,F,2001.0,WFM,1801,NaN,NaN,wi


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8553 entries, 0 to 8552
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Fide id          8553 non-null   int64  
 1   Name             8553 non-null   object 
 2   Federation       8553 non-null   object 
 3   Gender           8553 non-null   object 
 4   Year_of_birth    8261 non-null   float64
 5   Title            3118 non-null   object 
 6   Standard_Rating  8553 non-null   int64  
 7   Rapid_rating     3608 non-null   float64
 8   Blitz_rating     3472 non-null   float64
 9   Inactive_flag    5852 non-null   object 
dtypes: float64(3), int64(2), object(5)
memory usage: 668.3+ KB


### Preprocessing

In [6]:
df = data.copy()

In [7]:
df = df.drop(['Fide id', 'Name', 'Gender'], axis=1)

In [8]:
df

,Federation,Year_of_birth,Title,Standard_Rating,Rapid_rating,Blitz_rating,Inactive_flag
0,HUN,1976.0,GM,2675,2646.0,2736.0,wi
1,CHN,1994.0,GM,2658,2621.0,2601.0,NaN
2,IND,1987.0,GM,2586,2483.0,2483.0,NaN
3,RUS,1998.0,GM,2582,2502.0,2441.0,NaN
4,HUN,1969.0,GM,2577,NaN,NaN,wi
...,...,...,...,...,...,...,...
8548,BOL,NaN,NaN,1801,NaN,NaN,wi
8549,CZE,1994.0,NaN,1801,1791.0,1765.0,NaN
8550,IND,1992.0,NaN,1801,1884.0,1824.0,wi
8551,USA,2001.0,WFM,1801,NaN,NaN,wi


##### Missing values

In [9]:
df.isnull().sum()

Federation            0
Year_of_birth       292
Title              5435
Standard_Rating       0
Rapid_rating       4945
Blitz_rating       5081
Inactive_flag      2701
dtype: int64

In [10]:
df.dtypes

Federation          object
Year_of_birth      float64
Title               object
Standard_Rating      int64
Rapid_rating       float64
Blitz_rating       float64
Inactive_flag       object
dtype: object

In [13]:
numerical_features = ['Year_of_birth', 'Rapid_rating', 'Blitz_rating']

In [14]:
for column in numerical_features:
    df[column] = df[column].fillna(df[column].mean())

In [15]:
df.isnull().sum()

Federation            0
Year_of_birth         0
Title              5435
Standard_Rating       0
Rapid_rating          0
Blitz_rating          0
Inactive_flag      2701
dtype: int64

In [16]:
df['Title'].unique()

array(['GM', 'IM', 'WGM', 'FM', 'WFM', 'WIM', nan, 'CM', 'WCM', 'WH'],
      dtype=object)

In [17]:
df['Inactive_flag'].unique()

array(['wi', nan], dtype=object)

In [18]:
df['Inactive_flag'] = df['Inactive_flag'].fillna('wa')

In [19]:
df.isnull().sum()

Federation            0
Year_of_birth         0
Title              5435
Standard_Rating       0
Rapid_rating          0
Blitz_rating          0
Inactive_flag         0
dtype: int64

In [23]:
title_dummies = pd.get_dummies(df['Title'], dtype=int)
title_dummies

,CM,FM,GM,IM,WCM,WFM,WGM,WH,WIM
0,0,0,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
8548,0,0,0,0,0,0,0,0,0
8549,0,0,0,0,0,0,0,0,0
8550,0,0,0,0,0,0,0,0,0
8551,0,0,0,0,0,1,0,0,0


In [24]:
df = pd.concat([df, title_dummies['GM']], axis=1)

In [25]:
df

,Federation,Year_of_birth,Title,Standard_Rating,Rapid_rating,Blitz_rating,Inactive_flag,GM
0,HUN,1976.000000,GM,2675,2646.000000,2736.000000,wi,1
1,CHN,1994.000000,GM,2658,2621.000000,2601.000000,wa,1
2,IND,1987.000000,GM,2586,2483.000000,2483.000000,wa,1
3,RUS,1998.000000,GM,2582,2502.000000,2441.000000,wa,1
4,HUN,1969.000000,GM,2577,1931.680155,1925.155242,wi,1
...,...,...,...,...,...,...,...,...
8548,BOL,1985.291732,NaN,1801,1931.680155,1925.155242,wi,0
8549,CZE,1994.000000,NaN,1801,1791.000000,1765.000000,wa,0
8550,IND,1992.000000,NaN,1801,1884.000000,1824.000000,wi,0
8551,USA,2001.000000,WFM,1801,1931.680155,1925.155242,wi,0


In [26]:
df['GM'].unique()

array([1, 0])

In [27]:
df = df.drop('Title', axis=1)

In [28]:
df

,Federation,Year_of_birth,Standard_Rating,Rapid_rating,Blitz_rating,Inactive_flag,GM
0,HUN,1976.000000,2675,2646.000000,2736.000000,wi,1
1,CHN,1994.000000,2658,2621.000000,2601.000000,wa,1
2,IND,1987.000000,2586,2483.000000,2483.000000,wa,1
3,RUS,1998.000000,2582,2502.000000,2441.000000,wa,1
4,HUN,1969.000000,2577,1931.680155,1925.155242,wi,1
...,...,...,...,...,...,...,...
8548,BOL,1985.291732,1801,1931.680155,1925.155242,wi,0
8549,CZE,1994.000000,1801,1791.000000,1765.000000,wa,0
8550,IND,1992.000000,1801,1884.000000,1824.000000,wi,0
8551,USA,2001.000000,1801,1931.680155,1925.155242,wi,0


In [29]:
df.isnull().sum()

Federation         0
Year_of_birth      0
Standard_Rating    0
Rapid_rating       0
Blitz_rating       0
Inactive_flag      0
GM                 0
dtype: int64

#### Encoding

In [30]:
df['Inactive_flag'].unique()

array(['wi', 'wa'], dtype=object)

In [34]:
encoder = LabelEncoder()

df['Inactive_flag'] = encoder.fit_transform(df['Inactive_flag'])

In [35]:
df

,Federation,Year_of_birth,Standard_Rating,Rapid_rating,Blitz_rating,Inactive_flag,GM
0,HUN,1976.000000,2675,2646.000000,2736.000000,1,1
1,CHN,1994.000000,2658,2621.000000,2601.000000,0,1
2,IND,1987.000000,2586,2483.000000,2483.000000,0,1
3,RUS,1998.000000,2582,2502.000000,2441.000000,0,1
4,HUN,1969.000000,2577,1931.680155,1925.155242,1,1
...,...,...,...,...,...,...,...
8548,BOL,1985.291732,1801,1931.680155,1925.155242,1,0
8549,CZE,1994.000000,1801,1791.000000,1765.000000,0,0
8550,IND,1992.000000,1801,1884.000000,1824.000000,1,0
8551,USA,2001.000000,1801,1931.680155,1925.155242,1,0


In [36]:
df['Federation'].unique()

array(['HUN', 'CHN', 'IND', 'RUS', 'UKR', 'LTU', 'GEO', 'KAZ', 'IRI',
       'GER', 'SWE', 'BUL', 'TUR', 'GRE', 'AZE', 'FRA', 'ROU', 'USA',
       'MGL', 'POL', 'BLR', 'QAT', 'ESP', 'ENG', 'INA', 'ARM', 'CZE',
       'PER', 'SRB', 'NED', 'SCO', 'UZB', 'ITA', 'CUB', 'VIE', 'ECU',
       'AUS', 'ARG', 'CRO', 'SVK', 'SGP', 'ISR', 'LUX', 'SLO', 'EST',
       'CAN', 'LAT', 'AUT', 'SUI', 'MNC', 'MDA', 'BRA', 'BEL', 'COL',
       'PHI', 'PAR', 'BRU', 'MEX', 'BIH', 'MAS', 'NOR', 'MNE', 'TKM',
       'IRL', 'VEN', 'EGY', 'IRQ', 'FIN', 'BOL', 'DEN', 'MKD', 'KGZ',
       'ESA', 'CHI', 'RSA', 'FID', 'UAE', 'LBN', 'MYA', 'ISL', 'BAN',
       'POR', 'KSA', 'NAM', 'URU', 'ALG', 'WLS', 'PUR', 'ALB', 'KOR',
       'TJK', 'SRI', 'JAM', 'ANG', 'NGR', 'BAR', 'BER', 'ZIM', 'BOT',
       'JPN', 'DOM', 'CRC', 'SYR', 'GUA', 'SEY', 'JOR', 'NZL', 'MAR',
       'MAC', 'TTO', 'NCA', 'ZAM', 'PAN', 'THA', 'GCI', 'AHO', 'HKG',
       'MLT', 'HON', 'LBA', 'SUR', 'UGA', 'CPV', 'MAD'], dtype=object)

In [37]:
df.drop('Federation', axis=1, inplace=True)

In [38]:
df

,Year_of_birth,Standard_Rating,Rapid_rating,Blitz_rating,Inactive_flag,GM
0,1976.000000,2675,2646.000000,2736.000000,1,1
1,1994.000000,2658,2621.000000,2601.000000,0,1
2,1987.000000,2586,2483.000000,2483.000000,0,1
3,1998.000000,2582,2502.000000,2441.000000,0,1
4,1969.000000,2577,1931.680155,1925.155242,1,1
...,...,...,...,...,...,...
8548,1985.291732,1801,1931.680155,1925.155242,1,0
8549,1994.000000,1801,1791.000000,1765.000000,0,0
8550,1992.000000,1801,1884.000000,1824.000000,1,0
8551,2001.000000,1801,1931.680155,1925.155242,1,0


In [39]:
y = df['GM'].copy()
X = df.drop('GM', axis=1).copy()

In [40]:
y

0       1
1       1
2       1
3       1
4       1
       ..
8548    0
8549    0
8550    0
8551    0
8552    0
Name: GM, Length: 8553, dtype: int64

In [41]:
X

,Year_of_birth,Standard_Rating,Rapid_rating,Blitz_rating,Inactive_flag
0,1976.000000,2675,2646.000000,2736.000000,1
1,1994.000000,2658,2621.000000,2601.000000,0
2,1987.000000,2586,2483.000000,2483.000000,0
3,1998.000000,2582,2502.000000,2441.000000,0
4,1969.000000,2577,1931.680155,1925.155242,1
...,...,...,...,...,...
8548,1985.291732,1801,1931.680155,1925.155242,1
8549,1994.000000,1801,1791.000000,1765.000000,0
8550,1992.000000,1801,1884.000000,1824.000000,1
8551,2001.000000,1801,1931.680155,1925.155242,1


In [43]:
scaler = MinMaxScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [44]:
X

,Year_of_birth,Standard_Rating,Rapid_rating,Blitz_rating,Inactive_flag
0,0.622222,1.000000,1.000000,1.000000,1.0
1,0.822222,0.980549,0.982419,0.914394,0.0
2,0.744444,0.898169,0.885373,0.839569,0.0
3,0.866667,0.893593,0.898734,0.812936,0.0
4,0.544444,0.887872,0.497665,0.485831,1.0
...,...,...,...,...,...
8548,0.725464,0.000000,0.497665,0.485831,1.0
8549,0.822222,0.000000,0.398734,0.384274,0.0
8550,0.800000,0.000000,0.464135,0.421687,1.0
8551,0.900000,0.000000,0.497665,0.485831,1.0


### Training

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True, random_state=4)

In [54]:
model = LogisticRegression()
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [55]:
print(f"Model Accuracy: {model.score(X_test, y_test)}")

Model Accuracy: 0.994155464640561


In [56]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], shape=(1711,))

In [57]:
y_test

2695    0
6720    0
6056    0
7357    0
2524    0
       ..
236     0
731     0
6216    0
7081    0
7907    0
Name: GM, Length: 1711, dtype: int64

In [58]:
print(f"Model F1 Score: {f1_score(y_test, y_pred)}")

Model F1 Score: 0.16666666666666666


In [63]:
print(f"Percent GrandMasters: {y_test.sum()*100/len(y)}%")

Percent GrandMasters: 0.1286098444990062%
